In [14]:
#0读取数据 并 处理该小问异常值
import pandas as pd
import numpy as np

cost = pd.read_csv('各年7.1-7.7单品批发价格.csv')
sale_category = pd.read_csv('各年7.1-7.7各品类销量.csv')
sale_product = pd.read_csv('各年7.1-7.7单品销量.csv')
price = pd.read_csv('各年7.1-7.7单品售价.csv')
price_discount = pd.read_csv('各年7.1-7.7单品折扣价格.csv')
p1 = pd.read_csv('p1.csv')
p2 = pd.read_csv('p2.csv')
c_df = pd.read_csv('Prophet单品批发价格预测.csv')

cost = cost.set_index(cost['日期'])
cost = cost.drop('日期', axis=1)

sale_category = sale_category.set_index(sale_category['销售日期'])
sale_category = sale_category.drop('销售日期', axis=1)

sale_product = sale_product.set_index(sale_product['销售日期'])
sale_product = sale_product.drop('销售日期', axis=1)

price = price.set_index(price['销售日期'])
price = price.drop('销售日期', axis=1)

price_discount = price_discount.set_index(price_discount['销售日期'])
price_discount = price_discount.drop('销售日期', axis=1)

c_df = c_df.set_index(c_df.iloc[:,0])
c_df = c_df.drop(c_df.columns[0], axis=1)

#处理cost中的异常值，作为基准处理price与sale_product中的异常值
num_costna = cost.isna().sum()
num_costna = num_costna[num_costna != 21]
cost_processed = cost[num_costna.index]
price_processed = price[cost_processed.columns]
# price_discount_processed = price_discount[cost_processed.columns]
c_df = c_df[cost_processed.columns]
sale_product_processed = sale_product[cost_processed.columns]

#处理sale_category中的异常值
product_info = pd.read_excel('附件1.xlsx')
info_dict = product_info[['单品名称', '分类名称']].set_index('单品名称').to_dict(orient='dict')['分类名称']

zero = np.zeros((21,6))
sale_category_processed =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in sale_product_processed.columns:
    sale_category_processed[info_dict[col]] += sale_product_processed[col]

c_df

,七彩椒(1),七彩椒(2),上海青,东门口小白菜,云南油麦菜,云南生菜,云南生菜(份),余干椒,冰草,净藕(1),...,青杭椒(2),青梗散花,青线椒,青茄子(1),高瓜(1),鲜木耳(1),鲜木耳(份),黄心菜(1),黄心菜(2),黄白菜(2)
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2023-07-01,11.283125,17.223246,4.448322,3.558951,4.288465,4.892639,3.156615,9.001972,9.141936,7.593090,...,6.633472,6.327193,8.030885,2.451122,10.695603,7.450570,1.269304,3.342631,3.572313,4.398471
2023-07-02,11.555013,17.286523,4.577368,3.703236,4.343757,4.982323,3.160196,11.219541,9.274606,7.332504,...,6.788234,6.232115,8.045755,2.528307,10.729535,7.467691,1.282147,3.227073,3.531505,4.456539
2023-07-03,11.877639,17.571826,4.558968,3.543832,4.274410,4.952225,3.096889,12.319600,9.393054,7.025901,...,6.831326,6.231063,7.845660,2.493652,10.709710,7.468844,1.273445,3.214212,3.444150,4.449769
2023-07-04,11.420698,18.210313,4.559555,3.516277,4.295227,4.932869,3.020461,13.788977,9.360426,6.748812,...,6.916969,6.223693,7.936348,2.479481,10.753400,7.424200,1.285398,3.169600,3.391892,4.500709
2023-07-05,11.405392,18.708175,4.642817,3.744003,4.316992,4.958944,2.892764,14.328330,9.473411,6.433894,...,6.926025,6.105857,8.017064,2.529456,10.939844,7.369914,1.300645,3.174790,3.274633,4.473733
2023-07-06,11.631606,19.020030,4.736267,3.683155,4.248266,4.927594,2.929317,14.765293,9.323351,6.053643,...,7.135817,6.126682,7.856343,2.530739,10.909184,7.362216,1.299923,3.133978,3.299617,4.509486
2023-07-07,12.235706,19.603635,4.667127,3.635333,4.305486,4.672407,2.788980,13.550692,9.308037,5.701222,...,7.045419,6.119803,8.271865,2.452341,10.770628,7.423415,1.301105,3.050977,3.204841,4.465784


In [43]:
#1.计算S_gt

S_max = sale_category_processed.apply(lambda x:x.max())
S_min = sale_category_processed.apply(lambda x:x.min())

S_gt = (sale_category_processed - S_min) / (S_max - S_min)
S_gt

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
销售日期,,,,,,
2020-07-01,0.035020,0.649325,0.534132,0.548335,0.706815,0.438047
2020-07-02,0.030933,0.545266,0.458425,0.482229,0.560319,0.799339
2020-07-03,0.027826,0.593928,0.413310,0.559204,0.535410,0.634997
2020-07-04,0.044650,1.000000,0.741609,1.000000,0.763524,0.753677
2020-07-05,0.021434,0.877858,0.737066,0.871932,1.000000,0.926712
2020-07-06,0.000000,0.581589,0.418167,0.340544,0.458786,0.476180
2020-07-07,0.006687,0.388773,0.502066,0.480418,0.232267,0.788750
2021-07-01,0.094891,0.296490,0.211391,0.358950,0.402187,0.224630
2021-07-02,0.132134,0.169250,0.000000,0.406567,0.359975,0.277632


In [16]:
#2.计算w
import datetime
w_k = price_processed / cost_processed - 1
# 0不重要，对应的当日单品销量会为0
w_k = w_k.fillna(0)
w_prime = w_k * sale_product_processed

zero = np.zeros((21,6))
w_x_Θ =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in w_prime.columns:
    w_x_Θ[info_dict[col]] += w_prime[col]

w = w_x_Θ / sale_category_processed

t_list = []
for date in w.index:
    time = datetime.datetime.strptime(date, '%Y-%m-%d')
    t_list.append(time.day)
t_np = np.array(t_list)
t_norm = (t_np - t_np.min()) / (t_np.max() - t_np.min())
w['date_factor'] = t_norm
w

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌,date_factor
销售日期,,,,,,,
2020-07-01,0.706978,0.590086,0.510566,0.685442,0.820073,0.576699,0.000000
2020-07-02,0.710128,0.757272,0.591105,1.355157,0.744661,0.650298,0.166667
2020-07-03,0.782531,0.699971,0.598779,0.902886,0.623897,0.729150,0.333333
2020-07-04,0.688605,0.648614,0.553701,0.879172,0.737760,0.685781,0.500000
2020-07-05,0.845018,0.670352,0.512468,0.799566,0.670179,0.675019,0.666667
2020-07-06,0.834862,0.641013,0.533176,0.891419,0.729580,0.673781,0.833333
2020-07-07,0.831502,0.667985,0.590915,0.829822,0.834436,0.728714,1.000000
2021-07-01,0.683266,0.754930,0.548351,0.814191,0.880824,0.621025,0.000000
2021-07-02,0.719280,0.776297,0.701186,0.758584,1.059710,0.617144,0.166667


In [17]:
#3.模型拟合预测S_j
import numpy as np
import math
class MyLogisticRegression:
    def __init__(self):
        """
        Class contructor.
        
        :attr beta: weights vector of logistic regression.
        """
        self.beta = None

    def fx(self, X):
        """
        Calculate the value of f(x) given x.

        :param X: numpy.ndarray with a shape of (n, d), input data.
        :return:
            fx_value: numpy.ndarray with a length of n, output of f(x).
        """
        n = X.shape[0]
        fx_value = np.zeros(n)
        for i in range(n):
            data = self.beta@X[i].T
            fx_value[i] = data
        fx_value = 1/(1+np.exp(-fx_value))
        return fx_value

    def loss(self, fx_value, y):
        """
        Calculate the loss function given the calculated value f(x) and the true label y.

        :param fx_value: numpy.ndarray with a length of n, 
                         a vector of hypothesis function values on these samples, which is the output of the function fx
        :param y: numpy.ndarray with a length of n, 
                  a vector of the true labels of these samples
        :return:
            CELoss: a float value of the cross-entropy loss.
        """
        loss = 0
        n = fx_value.shape[0]
        for i in range(n):
            loss = loss + (-y[i])*math.log(fx_value[i])-(1-y[i])*math.log((1-fx_value[i]))
        loss = loss / n
        return loss

    def fit(self, X, y, n_iters = 10000, alpha = 0.01):
        """
        Train the model using gradient descent

        :param X: numpy.ndarray with a shape of (n*d), input data.
        :param y: numpy.ndarray with a length of n, the true labels of these samples
        :param n_iters: int, number of iterations
        :param alpha: float, learning rate
        :return:
            CELoss_list: list with a length of n_iters+1, 
                         contains the loss values before the gradient descent and after the gradient descent.
        """

        n, d = X.shape
        
        self.beta = np.zeros(d+1)
        #the first element in X_ is 1
        X_ = np.column_stack([np.ones(n), X])
        CELoss_list = [self.loss(self.fx(X_), y)]
        
        for i in range(n_iters):
            fx_value = self.fx(X_)
            offset = np.zeros(d+1)
            for j in range(n):
                offset = offset + (-y[j]/fx_value[j]+(1-y[j])/(1-fx_value[j]))*fx_value[j]*(1-fx_value[j])*X_[j]
            offset = offset / n
            self.beta = self.beta - offset #*alpha
            
            CELoss_list.append(self.loss(self.fx(X_), y))
        
        return CELoss_list

    def predict(self, X):
        """
        Predict the labels of input instances.

        :param X: numpy.ndarray with a shape of (n*d), input data.
        :return:
            y_hat: numpy.ndarray with a length of n, the predicted labels of these samples
        """
        n = X.shape[0]
        X = np.column_stack([np.ones(n), X])
        fx_value = self.fx(X)
        return fx_value

In [18]:
# initialize and train the model
my_model = MyLogisticRegression()

i = w.columns[0]
X = w[[i, 'date_factor']].values
y = S_gt[i].values

losses = my_model.fit(X, y)
for i, loss in enumerate(losses):
    if i%10 ==0:
        print("Step {}: Cross-Entropy Loss: {}".format(i, loss)) 

Step 0: Cross-Entropy Loss: 0.6931471805599453
Step 10: Cross-Entropy Loss: 0.5595020085029164
Step 20: Cross-Entropy Loss: 0.557970407623832
Step 30: Cross-Entropy Loss: 0.556815537655738
Step 40: Cross-Entropy Loss: 0.555913510586324
Step 50: Cross-Entropy Loss: 0.5551921547967171
Step 60: Cross-Entropy Loss: 0.5546009068431891
Step 70: Cross-Entropy Loss: 0.5541042926904609
Step 80: Cross-Entropy Loss: 0.5536773725596363
Step 90: Cross-Entropy Loss: 0.5533025634453623
Step 100: Cross-Entropy Loss: 0.5529674244627599
Step 110: Cross-Entropy Loss: 0.5526631130905328
Step 120: Cross-Entropy Loss: 0.5523833083749448
Step 130: Cross-Entropy Loss: 0.5521234591514348
Step 140: Cross-Entropy Loss: 0.55188025863104
Step 150: Cross-Entropy Loss: 0.5516512767944884
Step 160: Cross-Entropy Loss: 0.5514347029206337
Step 170: Cross-Entropy Loss: 0.5512291650633576
Step 180: Cross-Entropy Loss: 0.5510336033477005
Step 190: Cross-Entropy Loss: 0.5508471809448459
Step 200: Cross-Entropy Loss: 0.5506

In [19]:
#得到七天各自S预测模型
S_model = {}
for i in S_gt.columns:
    X = w[[i, 'date_factor']].values
    y = S_gt[i].values
    my_model = MyLogisticRegression()
    my_model.fit(X, y)
    S_model[i] = my_model

    x = np.zeros((1,2))
    x[0,0] = 1.2
    x[0,1] = 0.0
    S_model[i].predict(x)
    print(i, my_model.beta, S_model[i].predict(x))


水生根茎类 [ 0.87044213 -3.07242384  0.31215144] [0.05644082]
花叶类 [ 5.49034117 -7.43220486 -0.38073743] [0.03142249]
花菜类 [-0.34783207 -0.37234675  0.73931001] [0.31117149]
茄类 [ 0.62168268 -0.74610861 -0.08747892] [0.43201182]
辣椒类 [-0.55381326  0.79281676 -0.56631668] [0.59810293]
食用菌 [-5.83608669  9.24336588 -0.17446366] [0.99481069]


In [20]:
# x = np.zeros((1,2))
# x[0,0] = 1.2
# x[0,1] = 0.0
x = np.array([1.2, 0.0]).reshape(1,2)
S_model['花叶类'].predict(x)[0]

0.03142248901082407

In [21]:
#4.处理得到avg_discount
discount_temp1 = price_discount / price_processed
missing_col = [col for col in price_processed.columns if col not in price_discount.columns]
discount_temp1[missing_col] = 1
discount_k = discount_temp1[price_processed.columns].fillna(1)

discount_prime = discount_k * sale_product_processed

zero = np.zeros((21,6))
discount_x_Θ =  pd.DataFrame(zero, columns=sale_category.columns, index=sale_category.index)
for col in discount_prime.columns:
    discount_x_Θ[info_dict[col]] += discount_prime[col]

discount_df = discount_x_Θ / sale_category_processed
discount_df.loc['avg_discount'] = discount_df.apply(lambda x:x.sum()/21)
discount = discount_df.loc['avg_discount']
discount

水生根茎类    0.979524
花叶类      0.974655
花菜类      0.980625
茄类       0.997835
辣椒类      0.992385
食用菌      0.947664
Name: avg_discount, dtype: float64

In [30]:
#5.处理制作p1,p2的字典
p1_dict = p1.set_index('分类名称').to_dict(orient='dict')['0']
p2_dict = p2.set_index('分类名称').to_dict(orient='dict')['0']

{'水生根茎类': 0.0144534142703672,
 '花叶类': 0.0210318035158078,
 '花菜类': 0.0082996631173022,
 '茄类': 0.0020344895778648,
 '辣椒类': 0.0010031671900583,
 '食用菌': 0.1148167555404863}

In [23]:
#6.计算得到c

t_list = []
for date in sale_product_processed.index:
    time = datetime.datetime.strptime(date, '%Y-%m-%d')
    t_list.append(time.day)
t_np = np.array(t_list)
sale_product_processed['date'] = t_np
sale_product_temp = sale_product_processed.groupby('date').sum().values
sale_product_processed = sale_product_processed.drop('date', axis=1)

t_list = []
for date in sale_category_processed.index:
    time = datetime.datetime.strptime(date, '%Y-%m-%d')
    t_list.append(time.day)
t_np = np.array(t_list)
sale_category_processed['date'] = t_np
sale_category_temp = sale_category_processed.groupby('date').sum().values
sale_category_processed = sale_category_processed.drop('date', axis=1)

c_prime = c_df * sale_product_temp

zero = np.zeros((7,6))
c_x_Θ =  pd.DataFrame(zero, columns=sale_category_processed.columns, index=c_prime.index)
for col in c_prime.columns:
    c_x_Θ[info_dict[col]] += c_prime[col]

c = c_x_Θ / sale_category_temp
c

C:\Users\谢嘉楠\AppData\Local\Temp\ipykernel_14668\1456669332.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sale_product_processed['date'] = t_np


,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
Unnamed: 0,,,,,,
2023-07-01,9.787288,4.105093,8.475622,3.706847,7.086264,6.539818
2023-07-02,9.698288,4.131553,8.644676,3.635820,6.470582,7.066302
2023-07-03,6.429283,4.133861,8.519877,3.690829,6.306548,7.545625
2023-07-04,7.392045,4.162202,8.558640,3.449725,6.994952,7.128029
2023-07-05,7.048889,4.016360,8.286091,3.644148,7.157121,6.753155
2023-07-06,6.429663,4.122649,8.311134,3.700899,6.651031,7.540654
2023-07-07,5.857301,4.140400,8.339956,3.544302,6.680578,7.226703


In [24]:
#7.读取yita
l = pd.read_excel('附件4.xlsx')
l['平均损耗率(%)_小分类编码_不同值'] = l['平均损耗率(%)_小分类编码_不同值'] / 100
loss_rate = l[['小分类名称','平均损耗率(%)_小分类编码_不同值']].set_index('小分类名称').to_dict(orient='dict')['平均损耗率(%)_小分类编码_不同值']
loss_rate

{'花菜类': 0.1551,
 '水生根茎类': 0.1365,
 '花叶类': 0.1283,
 '食用菌': 0.09449999999999999,
 '辣椒类': 0.0924,
 '茄类': 0.0668}

In [26]:
result = pd.DataFrame(index= [i for i in range(7)], columns= sale_category_processed.columns)
result

,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
label = '水生根茎类'
day = 0
w = 1.2
S_max

19.78457353535324

In [46]:
#结果生成
import numpy as np
import random

label_list = sale_category_processed.columns 
day_list = [0,1,2,3,4,5,6]

for day in day_list:
    for label in label_list:
        # 设置参数：温度、最低温度、降温系数、解空间
        f_list = []
        x_dict = {}
        S_dict = {}
        for iter in range(20):
            T = 20000
            Tmin = 1e-4
            delta = 0.98
            bound = [0, 1.5]
            # 定义目标函数
            func = lambda w: (p1_dict[label] * (S_model[label].predict(np.array([w, day/6]).reshape(1,2))[0] * (S_max[label] - S_min[label]) + S_min[label]) * (1 + w) * c[label][day] +
                            p2_dict[label] * (S_model[label].predict(np.array([w, day/6]).reshape(1,2))[0] * (S_max[label] - S_min[label]) + S_min[label]) * (1 + w) * discount[label] * c[label][day] -
                            (S_model[label].predict(np.array([w, day/6]).reshape(1,2))[0] * (S_max[label] - S_min[label]) + S_min[label]) * c[label][day] / (1 - loss_rate[label]))
            # 初始值定义（也表示最优值）
            x = random.uniform(bound[0], bound[1])
            f = func(x)

            # epoch = 0
            while T > Tmin:
                # 产生新解及新函数值
                x_new = x + (random.random() * 2 - 1) * T
                while not 0 < x_new < 1.5:# 保证生成的解在解空间中，使得每次温度降低是有意义的
                    x_new = x + (random.random() * 2 - 1) * T
                f_new = func(x_new)
                # 计算增量
                if f_new > f:  # 以1的概率接受新解
                    x, f = x_new, f_new
                else:
                    if pow(np.e, (f - f_new) / T) > random.random():  # 根据Metropolis准则接受新解
                        x, f = x_new, f_new
                T *= delta
            f_list.append(f)
            x_dict[f] = x
            S_dict[f] = (S_model[label].predict(np.array([x, day/6]).reshape(1,2))[0] * (S_max[label] - S_min[label]) + S_min[label])/ (1 - loss_rate[label])
                # epoch += 1
            # print(f'函数的最大值为：{f},此时变量的值为：{x}')
        f_max = np.array(f_list).max()
        x_max = x_dict[f_max]
        S_j_max = S_dict[f_max]
        result.loc[day, label] = (f_max, x_max, S_j_max)

C:\Users\谢嘉楠\AppData\Local\Temp\ipykernel_14668\1200590487.py:38: RuntimeWarning: overflow encountered in double_scalars
  if pow(np.e, (f - f_new) / T) > random.random():  # 根据Metropolis准则接受新解


In [48]:
#保存问题二结果
result.to_excel('问题二加成参数与补货量最优方案.xlsx')